In [28]:
#%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it
import ipyparallel as ipp

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [4]:
# Create subdirectory /synthetic_data if it does not exist
path = "./synthetic_data"

# Check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

Folder ./synthetic_data already exists


In [5]:
# Setting up the cluster
cluster = ipp.Cluster(n=52)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(52)
rc.ids
dv = rc[:]

Using existing profile dir: '/Users/stigomholt/.ipython/profile_default'
Starting 52 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1739723267-1xx1', profile='default', controller=<running>, engine_sets=['1739723268'])>

  0%|          | 0/52 [00:00<?, ?engine/s]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [6]:
with dv.sync_imports():
    from random import normalvariate
    import numpy as np

importing normalvariate from random on engine(s)
importing numpy on engine(s)


In [7]:
a = 10
b = 5
dv.push({"a": a, "b": b})

<AsyncResult(_push): pending>

In [8]:
# To ensure the variables are on engine 0
rc[0]["a"]

10

In [9]:
rc[50]["b"]

5

### Calculate the initial sizes of all six population categories

In [10]:
%%px
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

In [11]:
%%px
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags
):
    # Assumed winter survival parameters for all six population categories
    w_survival = [
        normalvariate(0.8, 0.04),
        normalvariate(0.92, 0.03),
        normalvariate(0.93, 0.03),
        normalvariate(0.8, 0.04),
        normalvariate(0.87, 0.04),
        normalvariate(0.93, 0.03),
    ]
    if w_survival[0] >= 1.0:
        fraction_female_calves_surviving_winter = 1.0
    else:
        fraction_female_calves_surviving_winter = w_survival[0]

    if w_survival[1] >= 1.0:
        fraction_young_hinds_surviving_winter = 1.0
    else:
        fraction_young_hinds_surviving_winter = w_survival[1]

    if w_survival[2] >= 1.0:
        fraction_adult_hinds_surviving_winter = 1.0
    else:
        fraction_adult_hinds_surviving_winter = w_survival[2]

    if w_survival[3] >= 1.0:
        fraction_male_calves_surviving_winter = 1.0
    else:
        fraction_male_calves_surviving_winter = w_survival[3]

    if w_survival[4] >= 1.0:
        fraction_young_stags_surviving_winter = 1.0
    else:
        fraction_young_stags_surviving_winter = w_survival[4]

    if w_survival[5] >= 1.0:
        fraction_adult_stags_surviving_winter = 1.0
    else:
        fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [12]:
%%px
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [13]:
%%px
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors):

    # Assumed fertility parameters for young and adult hinds
    calves_per_young_hind = normalvariate(0.4, 0.05)

    calves_per_adult_hind = normalvariate(0.975, 0.008)
    if calves_per_adult_hind >= 1.0:
        calves_per_adult_hind = 1.0

    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [14]:
%%px
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1 or adult_stags < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [15]:
%%px
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [16]:
%%px
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [17]:
%%px
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Transform harvest percentages to numbers

In [18]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

### Scripts for L2_norm_seen_deer analysis

In [19]:
%%px
def L2_norm_seen_deer(
    population_log,
    empirical_observations,
    number_of_years,
):
    """
    Calculating the minimum L2 norm between synthetic data and
    reported data
    """
    import numpy as np
    # Establish the two time series to be compared
    seen_deer_e = np.array(empirical_observations)
    before_hunt_record = [x[18] for x in population_log]
    seen_deer_p = np.array(before_hunt_record)
    
    if seen_deer_p[-1] > 10:
        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(seen_deer_p, seen_deer_e)) / np.sum(
            np.multiply(seen_deer_p, seen_deer_p)
        )
    else:
        scaling_factor = 100.
    
    seen_deer_p_scaled = scaling_factor * seen_deer_p

    # Find minimum Residual Sum Square value
    L2_norm_min = (np.sum((seen_deer_p_scaled - seen_deer_e) ** 2))**(1/2)

    return scaling_factor, L2_norm_min

### Main ipy parallel script

In [20]:
@dv.remote(block=True)
def ipy_master():
    import numpy as np

    # Initial preparations
    array_top_hits = np.zeros((number_of_top_hits, number_of_years, 34))
    new_val = np.arange(10000, 10000 + number_of_top_hits, 1)
    scaling_factor = 500
    for i in range(array_top_hits.shape[0]):
        array_top_hits[i, -1, array_top_hits.shape[2] - 2] = scaling_factor
        array_top_hits[i, -1, array_top_hits.shape[2] - 1] = new_val[i]

    # Start going through states
    for state in init_chunk:
        # Run through for replicate number z
        for z in range(number_of_replicates):
            replicate_number = z

            population_log = []

            # Calculate size of each category in the initial population December 31 year -1
            (
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
            ) = calculate_initial_size_of_each_category(state)

            # Record initial sizes of the six categories
            female_calves_initially = female_calves_survivors
            young_hinds_initially = young_hinds_survivors
            adult_hinds_initially = adult_hinds_survivors
            male_calves_initially = male_calves_survivors
            young_stags_initially = young_stags_survivors
            adult_stags_initially = adult_stags_survivors

            # Run through all years in the time series for each initial state
            for j in range(number_of_years):
                (
                    total_population_in_spring,
                    number_of_migrants,
                    migration_percentage,
                    hinds_per_stag_before_hunting,
                    total_population_before_hunting,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_after_hunting,
                    hinds_per_stag_after_hunting,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ) = project_population_development(
                    number_harvested_female_calves[j],
                    number_harvested_young_hinds[j],
                    number_harvested_adult_hinds[j],
                    number_harvested_male_calves[j],
                    number_harvested_young_stags[j],
                    number_harvested_adult_stags[j],
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    state[2],
                    j,
                )

                new_record = [
                    state[0],
                    state[1],
                    state[2],
                    female_calves_initially,
                    young_hinds_initially,
                    adult_hinds_initially,
                    male_calves_initially,
                    young_stags_initially,
                    adult_stags_initially,
                    replicate_number,
                    first_year + j,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_in_spring,
                    total_population_before_hunting,
                    total_population_after_hunting,
                    hinds_per_stag_before_hunting,
                    hinds_per_stag_after_hunting,
                    migration_percentage,
                    number_of_migrants,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ]

                population_log.append(new_record)

            # Do L2_norm fitting of the number_of_years population trajectory
            if municipality == "Surnadal":
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs_farmland,
                    number_of_years,
                )
            else:
                scaling_factor, L2_value = L2_norm_seen_deer(
                    population_log,
                    seen_deer_obs,
                    number_of_years,
                )

            # Find max L2 norm in array_top_hits and exchange if better fit
            if L2_value < array_top_hits[np.argmax(array_top_hits[:, -1, -1]), -1, -1]:
                # Add scaling_factor and L2 value at the end of each record in population_log
                # and exchange number_of_years records
                for k in range(len(population_log)):
                    population_log[k].extend([scaling_factor, L2_value])
                    array_top_hits[
                        np.argmax(array_top_hits[:, -1, -1]), k, :
                    ] = population_log[k]

    return array_top_hits

In [21]:
%%time
    
# Number of replicates of each initial condition 
number_of_replicates = 1 #50 #30
number_of_top_hits = 2000 # from each engine
dv.push(dict(number_of_replicates = number_of_replicates, number_of_top_hits = number_of_top_hits))

# To ensure the variables are on engines 
#print(rc[0]['number_of_replicates'],rc[50]['number_of_top_hits']) 

# Using all six municipalties
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
data_municipality = [
    data_Averoy,
    data_Tingvoll,
    data_Surnadal,
    data_Sunndal,
    data_Vestnes,
    data_Laerdal,
]

for q in range(len(municipalities)):
    [
        municipality,
        first_year,
        last_year,
        number_of_years,
        years,
        seen_deer_obs,
        seen_deer_obs_forest,
        seen_deer_obs_farmland,
        hinds_per_stag_obs,
        total_harvest,
        total_quota,
        fraction_female_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_male_calves_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
        spring_counts,
    ] = data_municipality[q]
    
    dv.push(dict(municipality = municipality ,
            first_year = first_year,
            last_year = last_year,
            number_of_years = number_of_years,
            years = years,
            seen_deer_obs = seen_deer_obs,
            seen_deer_obs_forest = seen_deer_obs_forest,
            seen_deer_obs_farmland = seen_deer_obs_farmland,
            hinds_per_stag_obs = hinds_per_stag_obs,
            total_harvest = total_harvest,
            fraction_female_calves_harvested = fraction_female_calves_harvested,
            fraction_young_hinds_harvested = fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested = fraction_adult_hinds_harvested,
            fraction_male_calves_harvested = fraction_male_calves_harvested,
            fraction_young_stags_harvested = fraction_young_stags_harvested,
            fraction_adult_stags_harvested = fraction_adult_stags_harvested,
            spring_counts = spring_counts))

    # To ensure the variables are on engines 
    #print(rc[5]['years'],rc[45]['total_harvest']) 
    
    # Setting up all initial conditions to be run with calibrated init_pop
    # Make all combinations of the six initial conditions
    #initial_population = np.arange(init_pop_range[q][0],init_pop_range[q][1], 10)
    initial_population = np.arange(900, 4000, 10)
    #initial_hinds_per_stag = np.array([1.7]) # uses the inverse of the estimated stags_per_hind threshold
    initial_hinds_per_stag = np.arange(1.3, 2.2, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    stags_per_hind_threshold = np.array([0])  # not really initial condition, but initial premise

    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    dv.scatter("init_chunk", all_initial_states)
    # To ensure the variables are on engines 
    #check = rc[0]['init_chunk']
    #print("len(init_chunk):",len(check)) 
    
    # Transform reported hunting percentages for all years to numbers
    (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    ) = transform_harvested_percentages_to_numbers(
        total_harvest,
        fraction_female_calves_harvested,
        fraction_male_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
    )

    dv.push(dict(number_harvested_female_calves = number_harvested_female_calves,
        number_harvested_young_hinds = number_harvested_young_hinds,
        number_harvested_adult_hinds= number_harvested_adult_hinds,
        number_harvested_male_calves = number_harvested_male_calves,
        number_harvested_young_stags= number_harvested_young_stags,
        number_harvested_adult_stags = number_harvested_adult_stags))
    
    # To ensure the variables are on engines 
    #print(rc[10]['number_harvested_young_hinds'],rc[45]['number_harvested_adult_stags']) 
    
    # Getting array_top_hits from all engines in a 3d array that needs to be flattened
    array_top_hits = np.array(ipy_master())
    aggregated_arr = array_top_hits.reshape(-1, array_top_hits.shape[-1])
          

    # Make pandas dataframe for given municipality
    column_values = [
        "init_pop",
        "init_hps",
        "sh_threshold",
        "i_fcalves",
        "i_yhinds",
        "i_ahinds",
        "i_mcalves",
        "i_ystags",
        "i_astags",
        "rep_num",
        "obs_year",
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "spring_pop",
        "tot_pop_bh",
        "tot_pop_ah",
        "hps_bh",
        "hps_ah",
        "mig%",
        "num_migs",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
        "scaling",
        "L2_norm",
    ]
    pop_data = pd.DataFrame(data=aggregated_arr, columns=column_values)
    pop_data = pop_data.astype(
        {
            "init_pop": "int",
            "rep_num": "int",
            "obs_year": "int",
            "i_fcalves": "int",
            "i_yhinds": "int",
            "i_ahinds": "int",
            "i_mcalves": "int",
            "i_ystags": "int",
            "i_astags": "int",
            "f_calves": "int",
            "y_hinds": "int",
            "a_hinds": "int",
            "m_calves": "int",
            "y_stags": "int",
            "a_stags": "int",
            "spring_pop": "int",
            "tot_pop_bh": "int",
            "tot_pop_ah": "int",
            "num_migs": "int",
        }
    ).round(14)

    # Sort and trim the data frame
    #pop_data_sorted = pop_data.sort_values(by=["L2_norm"]).reset_index(drop=True)
    pop_data_sorted = pop_data.sort_values(['L2_norm','obs_year'],ascending=[True, True]).reset_index(drop=True)
        
    # Storing all top hits to allow variation in the number of top hits later on
    save_dir = "synthetic_data/"
    pop_data_sorted.to_pickle(
        save_dir + "top_hits_" + municipality + "_seen_deer_stoch_par_no_emig" + ".pkl"
    )


    print("Municipality = ", municipality)
    pd.set_option('display.max_columns', None)
    display(pop_data_sorted)
    print()
    print()

Municipality =  Averoy


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1260,1.5,0.0,126,102,501,126,96,306,0,2007,211,68,528,207,73,343,1134,1631,1433,1.439121,1.432113,0.0,0,0.792773,0.917514,0.946146,0.785785,0.883362,0.915369,0.383115,0.970559,0.000683,0.981065
1,1260,1.5,0.0,126,102,501,126,96,306,0,2008,202,134,478,204,134,345,1247,1728,1500,1.274952,1.276402,0.0,0,0.840893,0.924537,0.865763,0.854381,0.839474,0.900057,0.451682,0.988738,0.000683,0.981065
2,1260,1.5,0.0,126,102,501,126,96,306,0,2009,194,97,479,197,144,415,1315,1789,1530,1.251211,1.029924,0.0,0,0.859196,0.884753,0.915491,0.782206,0.853388,0.895179,0.398644,0.973899,0.000683,0.981065
3,1260,1.5,0.0,126,102,501,126,96,306,0,2010,192,92,438,190,118,512,1370,1823,1544,0.949518,0.840766,0.0,0,0.843953,0.879635,0.870419,0.801092,0.952686,0.978819,0.470776,0.988746,0.000683,0.981065
4,1260,1.5,0.0,126,102,501,126,96,306,0,2011,177,74,414,177,116,536,1341,1765,1495,0.850487,0.748233,0.0,0,0.744133,0.931868,0.883717,0.830723,0.813679,0.917537,0.488971,0.986593,0.000683,0.981065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1260,2.0,0.0,207,140,422,207,140,140,0,2019,198,108,488,201,145,682,1661,2158,1825,0.827712,0.721491,0.0,0,0.823034,0.950217,0.900012,0.847493,0.884983,0.955654,0.372286,0.964833,0.000626,1.184020
1767996,1260,2.0,0.0,207,140,422,207,140,140,0,2020,171,61,434,181,109,729,1636,2104,1686,0.749069,0.591215,0.0,0,0.802634,0.932202,0.899987,0.824485,0.940849,0.926206,0.428655,0.967061,0.000626,1.184020
1767997,1260,2.0,0.0,207,140,422,207,140,140,0,2021,154,40,345,144,103,727,1514,1948,1515,0.682016,0.464347,0.0,0,0.834209,0.886675,0.959264,0.795309,0.864992,0.907245,0.479910,0.980754,0.000626,1.184020
1767998,1260,2.0,0.0,207,140,422,207,140,140,0,2022,131,49,295,130,86,709,1355,1709,1401,0.582869,0.433943,0.0,0,0.742730,0.963757,1.000000,0.798833,0.897750,0.890568,0.424109,0.978848,0.000626,1.184020




Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2540,1.4,0.0,304,180,945,304,353,450,0,2007,420,150,978,398,185,698,2325,3287,2830,1.288037,1.278201,0.0,0,0.786327,0.967583,0.946590,0.811563,0.936017,0.972280,0.453879,0.986097,0.000158,0.220705
1,2540,1.4,0.0,304,180,945,304,353,450,0,2008,397,225,892,373,284,786,2544,3474,2960,1.122406,1.043602,0.0,0,0.772493,0.911377,0.902870,0.878798,0.917156,0.972366,0.424983,0.987056,0.000158,0.220705
2,2540,1.4,0.0,304,180,945,304,353,450,0,2009,355,206,893,351,255,891,2615,3508,2954,1.042462,0.958508,0.0,0,0.749692,0.915522,0.930383,0.835309,0.827198,0.931860,0.349350,0.987302,0.000158,0.220705
3,2540,1.4,0.0,304,180,945,304,353,450,0,2010,302,218,811,302,178,1001,2581,3391,2815,0.951728,0.872987,0.0,0,0.842683,0.952281,0.853898,0.715759,0.887916,0.946721,0.343170,0.972551,0.000158,0.220705
4,2540,1.4,0.0,304,180,945,304,353,450,0,2011,340,166,837,330,190,945,2502,3328,2809,0.957002,0.883787,0.0,0,0.826463,0.870770,0.965223,0.833351,0.851541,0.873268,0.378962,0.962824,0.000158,0.220705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2710,1.6,0.0,474,238,845,474,298,379,0,2019,344,183,882,336,209,1159,2861,3754,3115,0.860452,0.778228,0.0,0,0.812733,0.906105,0.916700,0.797314,0.853441,0.902422,0.327390,0.982724,0.000146,0.272659
1767996,2710,1.6,0.0,474,238,845,474,298,379,0,2020,354,139,850,327,176,1142,2783,3676,2991,0.850148,0.750302,0.0,0,0.758425,0.897627,0.966879,0.803753,0.834911,0.913090,0.378438,0.974243,0.000146,0.272659
1767997,2710,1.6,0.0,474,238,845,474,298,379,0,2021,315,159,812,302,159,1042,2633,3486,2791,0.900388,0.808602,0.0,0,0.793077,0.949190,0.981092,0.785211,0.912030,0.846871,0.260322,0.980788,0.000146,0.272659
1767998,2710,1.6,0.0,474,238,845,474,298,379,0,2022,320,122,799,337,197,1046,2558,3413,2823,0.843304,0.740764,0.0,0,0.711087,0.907853,0.986418,0.871265,0.885162,0.943241,0.471563,0.981760,0.000146,0.272659




Municipality =  Surnadal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2460,1.4,0.0,381,89,901,381,282,424,0,2007,335,241,833,330,228,609,2160,2977,2578,1.233850,1.281759,0.0,0,0.785104,0.927965,0.900689,0.759495,0.940739,0.970084,0.477598,0.957781,0.000219,0.373121
1,2460,1.4,0.0,381,89,901,381,282,424,0,2008,291,188,826,308,208,722,2263,3026,2545,1.089551,1.089134,0.0,0,0.778638,0.914082,0.838477,0.846496,0.925175,0.971228,0.386064,0.969095,0.000219,0.373121
2,2460,1.4,0.0,381,89,901,381,282,424,0,2009,316,156,781,313,165,730,2207,2999,2463,1.067903,1.046273,0.0,0,0.791486,0.913367,0.892624,0.754791,0.879474,0.901102,0.375064,0.986825,0.000219,0.373121
3,2460,1.4,0.0,381,89,901,381,282,424,0,2010,258,173,697,256,192,714,2150,2874,2294,1.004208,0.960887,0.0,0,0.796440,0.901392,0.876545,0.858620,0.865231,0.904620,0.382300,0.977935,0.000219,0.373121
4,2460,1.4,0.0,381,89,901,381,282,424,0,2011,293,142,719,289,144,678,1976,2678,2267,1.030007,1.046403,0.0,0,0.789579,0.897711,0.920213,0.792977,0.747229,0.875832,0.424318,0.990817,0.000219,0.373121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2020,1.6,0.0,272,226,680,272,294,272,0,2019,309,133,783,275,141,734,2217,3006,2378,1.063122,1.045438,0.0,0,0.766837,0.978880,0.936257,0.777008,0.859682,0.913271,0.428681,0.971257,0.000218,0.452031
1767996,2020,1.6,0.0,272,226,680,272,294,272,0,2020,281,152,728,273,122,729,2133,2887,2287,1.051110,1.034218,0.0,0,0.764592,0.933624,0.934553,0.804231,0.854697,0.948634,0.361625,0.969616,0.000218,0.452031
1767997,2020,1.6,0.0,272,226,680,272,294,272,0,2021,274,153,685,258,123,711,2093,2824,2206,1.051093,1.004963,0.0,0,0.842470,0.917788,0.954824,0.798115,0.896393,0.949434,0.379997,0.975403,0.000218,0.452031
1767998,2020,1.6,0.0,272,226,680,272,294,272,0,2022,266,182,650,248,120,689,1940,2602,2156,1.021579,1.028044,0.0,0,0.826108,0.884333,0.901926,0.710816,0.879310,0.938711,0.403620,0.982350,0.000218,0.452031




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1640,1.4,0.0,164,191,574,164,218,328,0,2007,244,89,631,239,97,430,1486,2074,1734,1.312572,1.367425,0.0,0,0.748207,0.929707,0.945988,0.869171,0.854863,0.954950,0.377786,0.958679,0.000109,0.148881
1,1640,1.4,0.0,164,191,574,164,218,328,0,2008,259,140,587,266,150,428,1529,2135,1831,1.237208,1.257785,0.0,0,0.695963,0.913997,0.939083,0.805134,0.813983,0.955720,0.379162,0.968943,0.000109,0.148881
2,1640,1.4,0.0,164,191,574,164,218,328,0,2009,218,170,545,220,161,456,1565,2112,1773,1.184590,1.157251,0.0,0,0.818772,0.903908,0.867886,0.781425,0.858154,0.886363,0.399098,0.973508,0.000109,0.148881
3,1640,1.4,0.0,164,191,574,164,218,328,0,2010,218,123,558,209,138,492,1577,2123,1740,1.121119,1.082567,0.0,0,0.804420,0.926294,0.917051,0.827983,0.875513,0.918403,0.344666,0.983510,0.000109,0.148881
4,1640,1.4,0.0,164,191,574,164,218,328,0,2011,247,130,592,236,123,508,1601,2195,1840,1.138215,1.144817,0.0,0,0.823411,0.918169,1.000000,0.822877,0.915038,0.914129,0.407825,0.979586,0.000109,0.148881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1410,2.0,0.0,204,140,527,204,126,206,0,2019,206,136,540,195,114,566,1657,2230,1759,1.082695,0.993865,0.0,0,0.839305,0.897524,0.920423,0.739601,0.844892,0.978766,0.406375,0.965494,0.000117,0.169660
1767996,1410,2.0,0.0,204,140,527,204,126,206,0,2020,187,100,515,174,97,596,1584,2119,1672,0.969821,0.887257,0.0,0,0.765586,0.901672,0.923356,0.807923,0.897015,0.959714,0.414799,0.971437,0.000117,0.169660
1767997,1410,2.0,0.0,204,140,527,204,126,206,0,2021,175,79,487,146,50,570,1520,2041,1509,1.004417,0.913553,0.0,0,0.859561,0.976326,0.974690,0.786911,0.896635,0.894259,0.353876,0.967466,0.000117,0.169660
1767998,1410,2.0,0.0,204,140,527,204,126,206,0,2022,163,58,441,152,34,540,1349,1811,1390,0.905003,0.868925,0.0,0,0.740728,0.905182,0.901199,0.793825,0.869782,0.961841,0.454292,0.975272,0.000117,0.169660




CPU times: user 59.1 s, sys: 1min 37s, total: 2min 36s
Wall time: 21min 22s


In [22]:
# stop cluster syncronously
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 64981, 'identifier': 'ipcontroller-1739723267-1xx1-64977'}
Stopping engine(s): 1739723268
engine set stopped 1739723268: {'engines': {'0': {'exit_code': 0, 'pid': 64994, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 65000, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 65001, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 65003, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 65005, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 65008, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 65011, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 65014, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 65019, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 65022, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 65029, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 65035, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 65041, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 65047, 'identifier': '13'}, '14': {'ex

In [23]:
cluster._is_running()

False

# Testing the sanity of the results

In [26]:
# Revoking top hits frames
save_dir = "synthetic_data/"
# With option for emigration
top_hits_Averoy_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Averoy_seen_deer_stoch_par_no_emig.pkl"
)
top_hits_Tingvoll_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Tingvoll_seen_deer_stoch_par_no_emig.pkl"
)
top_hits_Surnadal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Surnadal_seen_deer_stoch_par_no_emig.pkl"
)
top_hits_Sunndal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Sunndal_seen_deer_stoch_par_no_emig.pkl"
)
top_hits_Vestnes_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Vestnes_seen_deer_stoch_par_no_emig.pkl"
)
top_hits_Laerdal_seen_deer = pd.read_pickle(
    save_dir + "top_hits_Laerdal_seen_deer_stoch_par_no_emig.pkl"
)

top_hits_seen_deer = [
    top_hits_Averoy_seen_deer,
    top_hits_Tingvoll_seen_deer,
    top_hits_Surnadal_seen_deer,
    top_hits_Sunndal_seen_deer,
    top_hits_Vestnes_seen_deer,
    top_hits_Laerdal_seen_deer,
]

In [27]:
save_dir = "synthetic_data/"
municipalities = ["Averoy", "Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
for q in range(len(top_hits_seen_deer)):
    # Read through and test for consistent periodicity regarding years
    # which checks that there are no conflicts associated with the sorting of the data frames
    counter = 0
    for i in range(0, len(top_hits_seen_deer[q]), number_of_years):
        for j in range(number_of_years):
            if top_hits_seen_deer[q]["obs_year"][i + j] != (2007 + j):
                # print('i:', i, 'j:', j, 'obs_year value:', top_hits_seen_deer[q]['obs_year'][i+j])
                counter = counter + 1
    print("Number of troubles in " + municipalities[q] + ":", counter)

Number of troubles in Averoy: 0
Number of troubles in Tingvoll: 0
Number of troubles in Surnadal: 0
Number of troubles in Sunndal: 0
Number of troubles in Vestnes: 0
Number of troubles in Laerdal: 0


#pd.set_option("display.precision", 8)
pd.options.display.float_format = '{:.6f}'.format
top_hits_Averoy_seen_deer.dtypes
pd.set_option('display.max_columns', None)